In [5]:
!jupyter nbconvert --to=python AutoEncoder.ipynb

[NbConvertApp] Converting notebook AutoEncoder.ipynb to python
[NbConvertApp] Writing 1705 bytes to AutoEncoder.py


In [1]:
from keras import Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Dense
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# load dataset
# MNIST, for now
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_width, img_height, img_depth = x_train.shape[1], x_train.shape[2], 1

In [3]:
# Data preparation
x_train = x_train/256
x_test = x_test/256
x_train = x_train.reshape(len(x_train), img_width, img_height, img_depth)
x_test = x_test.reshape(len(x_test), img_width, img_height, img_depth)

In [4]:
input_img = Input(
    shape=(x_train.shape[1], x_train.shape[2], 1)
)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='relu', padding='same')(x)

W0715 16:26:34.296953 47691399429568 deprecation_wrapper.py:119] From /cluster/home/akozharin/venvs/keras-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 16:26:34.317446 47691399429568 deprecation_wrapper.py:119] From /cluster/home/akozharin/venvs/keras-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 16:26:34.321012 47691399429568 deprecation_wrapper.py:119] From /cluster/home/akozharin/venvs/keras-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 16:26:34.341071 47691399429568 deprecation_wrapper.py:119] From /cluster/home/akozharin/venvs/keras-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_p

In [5]:
autoencoder = Model(input_img, decoded)

In [6]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

W0715 16:26:35.020234 47691399429568 deprecation_wrapper.py:119] From /cluster/home/akozharin/venvs/keras-gpu/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0715 16:26:35.038868 47691399429568 deprecation_wrapper.py:119] From /cluster/home/akozharin/venvs/keras-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0715 16:26:35.044801 47691399429568 deprecation.py:323] From /cluster/home/akozharin/venvs/keras-gpu/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
NUM_EPOCHS = 100
autoencoder.fit(
    x_train, x_train, 
    epochs=NUM_EPOCHS, 
    batch_size=128, 
    shuffle=True, 
    validation_data=(x_test, x_test)
)

W0715 16:26:36.109094 47691399429568 deprecation_wrapper.py:119] From /cluster/home/akozharin/venvs/keras-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 60000 samples, validate on 10000 samples
Epoch 1/100
 9856/60000 [===>..........................] - ETA: 24s - loss: 0.3195

KeyboardInterrupt: 

## Save the model

In [12]:
FILENAME = 'example_save.h5'
autoencoder.save(FILENAME)